# MDA 2021
## Pyspark Sample Code
-----------------------------------------------------------------

## Setup
--------------------------------------------------

Let's setup Spark on your Colab environment.  Run the cell below!

In [ ]:
dic_3 = apriori(rdd_txt, 3, 100)
dic_2 = apriori(rdd_txt, 2, 100)
dic_1 = apriori(rdd_txt, 1, 100)

In [ ]:
# Calculating the confidence scores

for k in dic_2:
    ls = k.split("|")
    print(f"Confidence score of {ls} to {ls[0]} is ", dic_2[k]/dic_1[str(ls[1])])
    print(f"Confidence score of {ls} to {ls[1]} is ", dic_2[k]/dic_1[str(ls[0])])

Confidence score of ['ELE17451', 'GRO99222'] to ELE17451 is  0.1423841059602649
Confidence score of ['ELE17451', 'GRO99222'] to GRO99222 is  0.03329032258064516
Confidence score of ['ELE17451', 'ELE26917'] to ELE17451 is  0.10383944153577661
Confidence score of ['ELE17451', 'ELE26917'] to ELE26917 is  0.06141935483870968
Confidence score of ['ELE26917', 'GRO99222'] to ELE26917 is  0.1545253863134658
Confidence score of ['ELE26917', 'GRO99222'] to GRO99222 is  0.06108202443280977
Confidence score of ['GRO73461', 'DAI22896'] to GRO73461 is  0.13535684987694832
Confidence score of ['GRO73461', 'DAI22896'] to DAI22896 is  0.04580788450860633
Confidence score of ['ELE17451', 'SNA99873'] to ELE17451 is  0.12289966394623139
Confidence score of ['ELE17451', 'SNA99873'] to SNA99873 is  0.06606451612903226
Confidence score of ['ELE17451', 'GRO56989'] to ELE17451 is  0.18931297709923664
Confidence score of ['ELE17451', 'GRO56989'] to GRO56989 is  0.032
Confidence score of ['ELE17451', 'FRO78087']

In [ ]:
for k in dic_3:
    ls = k.split("|")
    query_1 = str(ls[0]) + "|" + str(ls[1])
    query_2 = str(ls[0]) + "|" + str(ls[2])
    query_3 = str(ls[1]) + "|" + str(ls[2])
    print(f"Confidence score of {ls} to {ls[0]} is ", dic_3[k]/dic_2[query_3])
    print(f"Confidence score of {ls} to {ls[1]} is ", dic_3[k]/dic_2[query_2])
    print(f"Confidence score of {ls} to {ls[2]} is ", dic_3[k]/dic_2[query_1])

Confidence score of ['DAI62779', 'ELE17451', 'SNA99873'] to DAI62779 is  0.421875
Confidence score of ['DAI62779', 'ELE17451', 'SNA99873'] to ELE17451 is  0.2660098522167488
Confidence score of ['DAI62779', 'ELE17451', 'SNA99873'] to SNA99873 is  0.07520891364902507
Confidence score of ['DAI62779', 'SNA45677', 'ELE17451'] to DAI62779 is  0.5241635687732342
Confidence score of ['DAI62779', 'SNA45677', 'ELE17451'] to SNA45677 is  0.09818941504178273
Confidence score of ['DAI62779', 'SNA45677', 'ELE17451'] to ELE17451 is  0.23383084577114427
Confidence score of ['FRO40251', 'DAI62779', 'ELE17451'] to FRO40251 is  0.13579387186629527
Confidence score of ['FRO40251', 'DAI62779', 'ELE17451'] to DAI62779 is  0.5118110236220472
Confidence score of ['FRO40251', 'DAI62779', 'ELE17451'] to ELE17451 is  0.29770992366412213
Confidence score of ['DAI62779', 'ELE17451', 'GRO30386'] to DAI62779 is  0.44086021505376344
Confidence score of ['DAI62779', 'ELE17451', 'GRO30386'] to ELE17451 is  0.289139633

# Question 4

In [17]:
df=spark.read.csv('Sample_Traffic.csv',header=True,schema=schema)
df.show(1)

+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|DEVICE_CODE|SYSTEM_ID|ORIGINE_CAR_KEY|FINAL_CAR_KEY|CHECK_STATUS_KEY|COMPANY_ID|      PASS_DAY_TIME|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|     200501|       81|       10477885|     10477885|               5|       161|2021-06-01 03:54:39|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
only showing top 1 row



In [18]:
rdd = df.rdd.map(lambda x : ((x[3], x[-1].day),x[0]))\
    .groupByKey()\
    .map(lambda x : (x[0],sorted(list(set(list(x[1]))))))\
    .filter(lambda x : len(x[-1]) < 20)\
    .map(lambda x : x[-1])


In [19]:
def apriori(rdd, n, threshold):
    if n == 1:
        lst = rdd.flatMap(lambda x : x)\
        .map(lambda x : (x,1)).reduceByKey(lambda x, y : x+y)\
        .filter(lambda x : x[-1] > threshold)\
        .map(lambda x : ([x[0]],x[-1]))\
        .collect()
        return wrap(lst)
    else:
        dic = apriori(rdd, n-1, threshold)
        lst = rdd.flatMap(lambda x : list(combinations(x, n)))\
        .filter(lambda x : test_ng(x, dic, n-1))\
        .map(lambda x : (x,1)).reduceByKey(lambda x, y: x+y)\
        .filter(lambda x : x[-1] > threshold)\
        .collect()
        return wrap(lst)

def make_query(x):
    query = ""
    for i in x:
        query += str(i) + "|"
    return query[:-1]

def test_ng(x, dic, n):
    lst = list(combinations(x,n))
    for i in lst:
        if dic.get(make_query(i)) == None:
            return False
    return True

def wrap(x): 
    dic = {}
    for item in x:
        query = ""
        for i in item[0]:
            query += str(i) + "|"
        query = query[:-1]
        dic[query] = item[-1]

    return dic

In [ ]:
# l = apriori(rdd, 3, 1000)
l
# 3m 25s

In [ ]:
# SON Algorithm

new_rdd = rdd.zipWithIndex()

def special_filter(dic, x):
    lst = list(dic.keys())
    res = []
    for ls in lst:
        if checking_instances(x[0], ls):
            res.append((ls,1))
    return res
def checking_instances(x, st):
    sp = st.split("|")
    a = True
    for s in sp:
        if int(s) not in x:
            a = False
    return a


def son(new_rdd, n, threshold):
    # 1st Pass
    rdd_1 = new_rdd.filter(lambda x : x[1]%3==0).map(lambda x : x[0])
    rdd_2 = new_rdd.filter(lambda x : x[1]%3==1).map(lambda x : x[0])
    rdd_3 = new_rdd.filter(lambda x : x[1]%3==2).map(lambda x : x[0])

    dic_1 = apriori(rdd_1, n, threshold/3)
    dic_2 = apriori(rdd_2, n, threshold/3)
    dic_3 = apriori(rdd_3, n, threshold/3)

    dic = dict(chain.from_iterable(d.items() for d in (dic_1, dic_2, dic_3)))
    
    # 2nd Pass

    return new_rdd\
    .map(lambda x : special_filter(dic, x))\
    .filter(lambda x : len(x) > 0)\
    .flatMap(lambda x : x).\
    reduceByKey(lambda x, y : x + y)\
    .filter(lambda x : x[-1] > threshold).collect()

# dic = son(new_rdd, 4, 1000)

In [ ]:
son(new_rdd, 4, 1000)

[('900215|900234|900256|22010118', 1341),
 ('900102|900142|900212|900244', 1799),
 ('900139|900212|900244|100700826', 1315),
 ('900193|900212|900244|100700853', 1071),
 ('900142|900202|900212|900244', 1017),
 ('900142|900152|900212|900244', 1549),
 ('231|900236|900255|100700841', 1307),
 ('231|900101|900236|100700841', 1076),
 ('900212|900244|100700839|100700853', 1299),
 ('900142|900152|900212|100700853', 1299),
 ('900142|900212|900273|100700853', 1044),
 ('631633|900142|900212|900244', 1281),
 ('22010087|22010088|22010094|22010095', 2083),
 ('142|900215|900234|900256', 1055),
 ('900102|900212|900244|100700853', 1134),
 ('900142|900212|900244|900249', 1309),
 ('209103|900265|100700804|100700834', 1276),
 ('900101|900212|900244|100700839', 1105),
 ('900142|900193|900212|900244', 1126),
 ('900101|900212|900244|100700841', 1467),
 ('900139|900212|900244|100700839', 1093),
 ('900152|900212|900244|100700853', 1264),
 ('900102|900142|900212|100700853', 1088),
 ('900142|900212|900244|1007008